In [12]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [13]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [14]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [15]:
import pandas as pd
import numpy as np
import gzip

In [16]:
# load the 2017 data
scripts = pd.read_csv("dw-data/201701scripts_sample.csv.gz")
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [17]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [18]:
CHEM = pd.read_csv("dw-data/chem.csv.gz")
CHEM.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [9]:
def describe(key):
    desc = scripts[key].describe()
    total = desc['count'] * desc['mean']

    return (total, desc['mean'], desc['std'], desc['25%'], desc['50%'], desc['75%'])



summary_stats = [('items', describe('items')),('quantity', describe('quantity')), ('nic', describe('nic')), ('act_cost', describe('act_cost'))]

In [10]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [11]:
grpd_df = scripts.groupby('bnf_name')['items'].sum()

max_n = grpd_df.max()

most_common_item = [(grpd_df.index[ grpd_df == max_n][0], max_n)]

In [12]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [ ]:
group1=practices.groupby('code')['post_code'].min().reset_index()

group2=scripts.merge(group1, left_on='practice', right_on='code')

sum_bnf = group2.groupby(['post_code','bnf_name'])['items'].sum().reset_index()

sum_bnf_name = sum_bnf.sort_values('items', ascending = False).groupby('post_code').first()

sum_overall = group2.groupby('post_code')['items'].sum().reset_index()

merged_q3 = sum_bnf_name.merge(sum_overall, on = 'post_code')

merged_q3['proportion'] = merged_q3['items_x'] / merged_q3['items_y']

merged_Q3 = merged_q3[['post_code', 'bnf_name', 'proportion']].head(100)

In [ ]:
items_by_region = [row[1:] for row in merged_Q3.itertuples()]

In [ ]:
grader.score.dw__items_by_region(items_by_region)

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [36]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [37]:
opioids_join = '|'.join(opioids)
opioids_join

'morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine'

In [38]:
chem = pd.read_csv(gzip.open('./dw-data/chem.csv.gz', 'r'))
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [39]:
opioid_codes = chem.loc[chem['NAME'].str.contains(opioids_join, case=False)]['CHEM SUB'].tolist()
len(opioid_codes)

35

In [40]:
chem_new = chem
chem_new.columns = ['bnf_code','chem_name']
chem_new.head()

,bnf_code,chem_name
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [41]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [42]:
scripts.shape

(973193, 7)

In [43]:
scrpt = scripts
scrpt.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [44]:

scrpt['opioid_prescription'] = scrpt['bnf_code'].isin(opioid_codes)

In [45]:
scrpt.shape


(973193, 8)

In [46]:
scrpt.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [47]:
pract = practices[['code', 'name']]
pract.columns = ['practice', 'name']
pract.head()

,practice,name
0,A81001,THE DENSHAM SURGERY
1,A81002,QUEENS PARK MEDICAL CENTRE
2,A81003,VICTORIA MEDICAL PRACTICE
3,A81004,WOODLANDS ROAD SURGERY
4,A81005,SPRINGWOOD SURGERY


In [48]:
len(scrpt.loc[scrpt['opioid_prescription']])

34843

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [49]:
opioids_per_practice = scrpt.groupby('practice')['opioid_prescription'].mean().rename('frac')
opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: frac, dtype: float64

In [50]:
opioids_per_practice_std = scrpt.groupby('practice')['opioid_prescription'].std().rename('frac_std')
opioids_per_practice_std.head()

practice
A81005    0.179162
A81007    0.203666
A81011    0.210753
A81012    0.202466
A81017    0.191578
Name: frac_std, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [51]:
overall_rate = scrpt['opioid_prescription'].mean()
overall_rate


0.03580276471367961

In [52]:
overall_rate_std = scrpt['opioid_prescription'].std()
overall_rate_std

0.18579817605238425

In [53]:
relative_opioids_per_practice = (opioids_per_practice - overall_rate).rename('relative')
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: relative, dtype: float64

In [54]:
opioid = scrpt.groupby('practice')['opioid_prescription'].sum().rename('opioid')
opioid.head()

practice
A81005    50.0
A81007    63.0
A81011    73.0
A81012    57.0
A81017    82.0
Name: opioid, dtype: float64

In [55]:
total = scrpt.groupby('practice')['bnf_code'].count().rename('total')
total.head()

practice
A81005    1507
A81007    1454
A81011    1568
A81012    1332
A81017    2150
Name: total, dtype: int64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [56]:
standard_error_per_practice = (overall_rate_std/(total**0.5)).rename('std_err')
standard_error_per_practice.head()

practice
A81005    0.004786
A81007    0.004873
A81011    0.004692
A81012    0.005091
A81017    0.004007
Name: std_err, dtype: float64

In [59]:

opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_scores')
opioid_scores.head()

practice
A81005   -0.548306
A81007    1.544557
A81011    2.291795
A81012    1.373060
A81017    0.583168
Name: opioid_scores, dtype: float64

In [60]:
merged = pd.concat([opioid, total, opioids_per_practice, relative_opioids_per_practice, standard_error_per_practice,opioid_scores], axis=1)

In [61]:
merged = merged.reset_index()

In [62]:
merged.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores
0,A81005,50.0,1507,0.033179,-0.002624,0.004786,-0.548306
1,A81007,63.0,1454,0.043329,0.007526,0.004873,1.544557
2,A81011,73.0,1568,0.046556,0.010753,0.004692,2.291795
3,A81012,57.0,1332,0.042793,0.006990,0.005091,1.373060
4,A81017,82.0,2150,0.038140,0.002337,0.004007,0.583168


In [63]:
type(merged)

pandas.core.frame.DataFrame

In [64]:
merged.shape

(856, 7)

In [65]:
pract.reset_index().head()

,index,practice,name
0,0,A81001,THE DENSHAM SURGERY
1,1,A81002,QUEENS PARK MEDICAL CENTRE
2,2,A81003,VICTORIA MEDICAL PRACTICE
3,3,A81004,WOODLANDS ROAD SURGERY
4,4,A81005,SPRINGWOOD SURGERY


In [66]:
type(pract)

pandas.core.frame.DataFrame

In [67]:
final_df = merged.merge(pract, on='practice', how='left')

In [68]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
0,A81005,50.0,1507,0.033179,-0.002624,0.004786,-0.548306,SPRINGWOOD SURGERY
1,A81007,63.0,1454,0.043329,0.007526,0.004873,1.544557,BANKHOUSE SURGERY
2,A81011,73.0,1568,0.046556,0.010753,0.004692,2.291795,CHADWICK PRACTICE
3,A81012,57.0,1332,0.042793,0.006990,0.005091,1.373060,WESTBOURNE MEDICAL CENTRE
4,A81017,82.0,2150,0.038140,0.002337,0.004007,0.583168,WOODBRIDGE PRACTICE


In [69]:

final_df.sort_values('opioid_scores', ascending = False , inplace=True)

In [70]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [57]:
def script_anomalies():
    return [("ADDACTION NORTH DEVON", 10.5738246391, 4.0)] * 100

In [71]:
final = final_df.drop_duplicates('name')
final.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [72]:

result = final[['name','opioid_scores','total']]
result.head()

,name,opioid_scores,total
829,NATIONAL ENHANCED SERVICE,11.695818,7
874,OUTREACH SERVICE NH / RH,7.339043,2
895,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
296,H&R P C SPECIAL SCHEME,5.123032,36
948,HMR BARDOC OOH,4.958866,321


In [73]:
result = result.head(100)
values = result.get_values().tolist()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.values' or 'np.asarray(..)' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
answer=[]

for item in values:
    answer.append(tuple(item))

In [75]:
def script_anomalies():
    return answer

In [76]:
grader.score('dw__script_anomalies', script_anomalies)

Your score:  0.9400000000000006


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
unique_practices = practices.sort_values(by='name').groupby('code', sort=False).first()
unique_practices.reset_index(inplace=True)

final_merge = opioid_scores.merge(unique_practices[['code','name']], left_on='practice',
                                  right_on='code', how='left')

results = final_merge.merge(bnf_by_practice.reset_index(), on='practice')

subset = results[['name', 'opioid_scores', 'bnf_sum']]
anomalies = [tuple(val) for val in subset.values]
anomalies = sorted(anomalies, key=lambda x: x[1], reverse=True)[:100]

In [ ]:
results.head()

In [ ]:
grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [ ]:
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()

In [ ]:
resp = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()

In [ ]:
resp.fillna(0, inplace=True)

In [ ]:
resp = resp[resp['item16'] >= 50]

In [ ]:
resp.sort_values(by='growth', ascending = False, inplace=True)

In [ ]:
script_growth = list(pd.concat([resp.head(50), resp.tail(50)]).itertuples(index=False, name=None))

In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
p = 1/scripts.bnf_code.nunique()

rates = scripts['bnf_code'].value_counts() / len(scripts)

mask = rates < .1 * p

rare_codes = rates[mask].index.unique()

scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/scripts.groupby('practice')['act_cost'].sum()).fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = (rare_cost_prop - scripts[scripts['rare']]['act_cost'].sum() /scripts['act_cost'].sum())

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_error = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = relative_rare_cost_prop/standard_error
unique_practices = (practices.sort_values('name').groupby('code').first().reset_index())[['code', 'name']].rename(columns={'code':'practice'})

unique_practices.set_index('practice', inplace=True)

rare_scores = pd.concat([unique_practices, rare_scores], axis =1).sort_values(by='act_cost', ascending=False).head(100)

In [ ]:
rare_scripts = list(rare_scores.itertuples(name=None))

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*